# Diabetes Detection Demo

This notebook demonstrates how to use the Diabetes Detection system.

## 1. Setup and Imports

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Add src to path
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from diabetes_detector import DiabetesDetector

## 2. Load and Explore Data

In [ ]:
# Initialize detector
detector = DiabetesDetector(model_type='logistic')

# Load data
X, y = detector.load_data('data/sample_diabetes.csv')

print(f"Dataset shape: {X.shape}")
print(f"\nFeatures: {', '.join(detector.feature_names)}")
print(f"\nTarget distribution:\n{y.value_counts()}")

In [ ]:
# Display first few rows
df = pd.concat([X, y], axis=1)
df.head()

## 3. Train the Model

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Preprocess data
X_train_scaled, y_train = detector.preprocess_data(X_train, y_train, fit_scaler=True)
X_test_scaled = detector.preprocess_data(X_test, fit_scaler=False)

# Train model
detector.train(X_train_scaled, y_train)

print("Model trained successfully!")

## 4. Evaluate the Model

In [ ]:
# Evaluate on test set
metrics = detector.evaluate(X_test_scaled, y_test)

print(f"Test Accuracy: {metrics['accuracy']:.4f}")
print(f"\nConfusion Matrix:\n{metrics['confusion_matrix']}")
print(f"\nClassification Report:\n{metrics['classification_report']}")

## 5. Make Predictions

In [ ]:
# Example 1: High-risk patient
patient_1 = np.array([[6, 148, 72, 35, 0, 33.6, 0.627, 50]])
patient_1_scaled = detector.scaler.transform(patient_1)

pred_1 = detector.predict(patient_1_scaled)[0]
prob_1 = detector.predict_proba(patient_1_scaled)[0]

print("Patient 1 (High Risk):")
print(f"  Prediction: {'DIABETES' if pred_1 == 1 else 'NO DIABETES'}")
print(f"  Probability of Diabetes: {prob_1[1]:.2%}")

In [ ]:
# Example 2: Low-risk patient
patient_2 = np.array([[1, 85, 66, 29, 0, 26.6, 0.351, 31]])
patient_2_scaled = detector.scaler.transform(patient_2)

pred_2 = detector.predict(patient_2_scaled)[0]
prob_2 = detector.predict_proba(patient_2_scaled)[0]

print("Patient 2 (Low Risk):")
print(f"  Prediction: {'DIABETES' if pred_2 == 1 else 'NO DIABETES'}")
print(f"  Probability of Diabetes: {prob_2[1]:.2%}")

## 6. Save the Model

In [ ]:
# Save model for later use
detector.save_model('models/diabetes_model.pkl')

## 7. Load and Use Saved Model

In [ ]:
# Create a new detector and load the saved model
new_detector = DiabetesDetector()
new_detector.load_model('models/diabetes_model.pkl')

# Make a prediction with the loaded model
test_patient = np.array([[3, 120, 70, 30, 135, 42.9, 0.452, 30]])
test_patient_scaled = new_detector.scaler.transform(test_patient)

prediction = new_detector.predict(test_patient_scaled)[0]
probability = new_detector.predict_proba(test_patient_scaled)[0]

print("Test Patient:")
print(f"  Prediction: {'DIABETES' if prediction == 1 else 'NO DIABETES'}")
print(f"  Probability of Diabetes: {probability[1]:.2%}")